In [ ]:
'''
Reference:
Example script from Keras Github Repository keras/examples/lstm_text_generation.py
https://github.com/keras-team/keras

Same configuration as the vanilla LSTM char-RNN for Donna Summer, except with 512 hidden units and 
170 characters as the length of each block of sequence
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

#path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
#text = io.open(path, encoding='utf-8').read().lower()
import pandas as pd
lyrics_data = pd.read_csv('songdata.csv')

# Donna Summer

lyrics_data_donna = lyrics_data[lyrics_data.artist == 'Donna Summer']['text']

lyrics_concat_donna = u''
for i in lyrics_data_donna.index:
    lyrics_concat_donna += lyrics_data_donna[i]
    
print("input chars length")
print(len(lyrics_concat_donna))
text = lyrics_concat_donna
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 170
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, input_shape=(maxlen, len(chars))))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Using TensorFlow backend.


input chars length
236908
corpus length: 236908
total chars: 73
nb sequences: 78913
Vectorization...
Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
78913/78913 [==============================] - 173s - loss: 2.7670   

----- diversity: 0.2
----- Generating with seed: " 
And I want, exactly what he's got  
And I don't ever mind giving him every  
Bit of my love  
And baby you know  
I've got a lot to give  
I got fire in my touch  
Ther"
 
And I want, exactly what he's got  
And I don't ever mind giving him every  
Bit of my love  
And baby you know  
I've got a lot to give  
I got fire in my touch  
There oustound out out out of of out a me foull out a foull out out out out me around oustound out out me tound oustound out a found ou tound out in to to four out a fout out foull out out ouston't ous out me out and out out out alound oustound of a fould oustouthe meous out oustound out out out out me you  
I foull oust oustous out out out out out 

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in log


u I know I know I know I know I know I want to believe  
I know I know I know I know I know I want to believ  
I know I know I know I know I know I know I know I know I know I want to be to 

----- diversity: 0.5
----- Generating with seed: "baby tell me what it is  
Something baby is blowing in the wind  
Out in the wind  
  
Nothing has changed  
That's what you say  
So how come each date  
You turn up lat"
baby tell me what it is  
Something baby is blowing in the wind  
Out in the wind  
  
Nothing has changed  
That's what you say  
So how come each date  
You turn up late we'll see  
Some will there  
I want to start my stop to be for a life  
You want to time  
There spol the bome to be to me  
I pelling for a rool  
In a lettle friends out  
  
In a fairy the walk and the wall  
When it see time  
There we'll believ  
  
ruch to sing on the walk  
I want to the believe  
  
It's always believe to me  
So in there in my song  
I want to give in the time  
And I 

----- diversit

We've got love, happily ever after  
It's the mighty be the way  
It's a wanderer  
It's a wanderer  
And I'm were the ment of my heart  
And she world take is heart  
And she works hard  
The want to be the way  
The was shile of heart  
The want to be the way  
There's not the way  
There's not the way  
There's so mattin'  
It's someone's we can't go on the way  
There's not the way  
It's the still be the way  
It's over  
It's

----- diversity: 0.5
----- Generating with seed: "there's nobody there  
We're never alone, we got love to share  
Lots of love, happily ever after  
Lost of love, happily ever after  
We've got love, happily ever after "
there's nobody there  
We're never alone, we got love to share  
Lots of love, happily ever after  
Lost of love, happily ever after  
We've got love, happily ever after  
Still you believe the moon  
There's somebody, it's one the starts on the bestect my knows  
And starts the tanes of my heart  
The way heart  
There's not is the start 

...

I got w                      1   1            1           1   1          1               1                      1       1                                 1                                                                                                                                                                                                                                                            

----- diversity: 1.2
----- Generating with seed: "Now, your mama won't like it when she finds out  
Girl is out at night  
  
Hey, mister, have you got a dime?  
Mister, do you want to spend some time? Oh, yeah  
I got w"
Now, your mama won't like it when she finds out  
Girl is out at night  
  
Hey, mister, have you got a dime?  
Mister, do you want to spend some time? Oh, yeah  
I got w                      1   1            1           1   1          1               1                      1       1                                 1                                                   

Rouge o                                                     1                                               1        1             1                                                                                                                                                                                                                                                                                   

----- diversity: 0.5
----- Generating with seed: "baby from your love  

Queen for a day, queen for a night  
Dressed head to toe, so you'd never know it's me, it's me  
  
Ribbons and lace, shoes made of glass  
Rouge o"
baby from your love  

Queen for a day, queen for a night  
Dressed head to toe, so you'd never know it's me, it's me  
  
Ribbons and lace, shoes made of glass  
Rouge o                                                     1                                               1        1             1                                                                          

I've never shown what I've showi                 1                          1                 1                             1        1             1                                                     1                                                                                                                                                                                                                                      

----- diversity: 1.2
----- Generating with seed: " to be the woman in me  
It's so hard to believe that I'm feeling so free  
To be the woman in me  
Like the dark side of the full moon  
I've never shown what I've showi"
 to be the woman in me  
It's so hard to believe that I'm feeling so free  
To be the woman in me  
Like the dark side of the full moon  
I've never shown what I've showi                 1                          1                 1                             1        1             1                                                     1    

Oh no, it's  1                         1                   1      1                            1                         1                   1      1                            1                                                                                                                                                                                                                                         

----- diversity: 0.5
----- Generating with seed: ", it's not the way  
Oh no, it's not the way  
Listen mister to what I say  
Oh no, it's not the way  
Oh no, it's not the way  
Listen mister to what I say  
Oh no, it's"
, it's not the way  
Oh no, it's not the way  
Listen mister to what I say  
Oh no, it's not the way  
Oh no, it's not the way  
Listen mister to what I say  
Oh no, it's  1                         1                   1      1                            1                         1                   1      1                            1                            

I'll go wher                       1                         1   1   1            1                                               1       1             1       1                                                                                                                                                                                                                                                           

----- diversity: 1.2
----- Generating with seed: "ou're my breath  
The air I breathe  
You are my imagination  
Su le finesse  
Song of my heart  
Secrets I'm not afraid to whisper  
  
I will go with you  
I'll go wher"
ou're my breath  
The air I breathe  
You are my imagination  
Su le finesse  
Song of my heart  
Secrets I'm not afraid to whisper  
  
I will go with you  
I'll go wher                       1                         1   1   1            1                                               1       1             1       1                                             

In [ ]:
# sentence 1

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'Loneliness is always looking for a frien'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 2

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "We go hard or we go home \nWe gon' do thi"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 3

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "At first I was afriad, I was petrified \n"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 4

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Young Money, Nicki Minaj, Justin  \n  \nSh"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 5

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Life is to be lived on the positive tip "
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 6

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Another night I'm waiting for you  \nWait"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 7

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'Back track, listen, stop that  \nBefore e'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 8

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'How can a non-musician discuss the futur'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 9

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'Hi out there. This is Shaun Goldberg and'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# sentence 10

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Whose eyes in my behind  \nI don't recogn"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()